# Dense retrieval + bert(rerank)

작성일자: 2101\
작성자: 조진욱\
목표: dpr 같은 모델을 만들자
순서: 
3-1 과 동일

비고:
1. load 함수에서 json 형태로 불러오도록 되어있어 필요없지만 retrieval 의 결과를 json 으로 저장하는 과정을 거침. 
2. reader(mrc) 모델은 encoder 모델에서 topk 를 뽑은 뒤 합쳐서 rerank 만 도입한 것. 구현하기 쉬우나 학습하는데 시간이 걸려서 일단 사전 훈련된 모델을 가져옴 
3. 그러나 1-2 best metric 보다 성능이 안나와야 정상. 왜냐하면 retrieval 과정에서 잘못된 context 들이 선택되었을 것이기 때문.

## Step1 retrieval 모델 불러와서 각 query 에 대한 document 찾기 
현재 dense retrieval은 기본값으로 하나의 query 당 5개의 doc을 불러오도록 해뒀으나, 우리는 mrc 형식으로 best 1개 document 만을 선택하도록 할 것이므로 n_doc 값을 1로 조정해야함

In [7]:
import os
from retrieval import DenseRetrieval
from utils import save_json


In [4]:
mode = 'dev'

In [6]:
model_name = 'facebook/dpr-question_encoder-single-nq-base'
qas_file =  f'{mode}_qa.json'
passages_path = os.path.join(cfg.dense_dir, "dpr_dataset")
index_path = os.path.join(cfg.dense_dir, "dpr_dataset_hnsw_index.faiss")
ret_ds = DenseRetrieval.from_pretrained(model_name,
                                        passages_path=passages_path,
                                        index_path=index_path,
                                       mode=mode)
cqa_df = ret_ds.retrieve(qas_file)
res_path = os.path.join(cfg.dense_dir, f"retrived_{mode}_dense.csv")
cqa_df.to_csv(res_path, sep='\t')

searching indexed dataset
passages and index is exist


In [9]:
result_filename = f"retrived_{mode}_dense.json"
save_json(cqa_df, result_filename)

KeyError: 'q_id'

In [ ]:
## Step2 bert 모델이 만들어진 json 파일을 다시 불러와 torch Dataset의 형식으로 변환, mrc 형식으로 각 인스턴스에 대한 start end position 을 구한뒤 그에 대한 평가 진행 EM, F1_score

In [ ]:
from transformers import DPRReader, DPRReaderTokenizer
tokenizer = DPRReaderTokenizer.from_pretrained('facebook/dpr-reader-single-nq-base')
model = DPRReader.from_pretrained('facebook/dpr-reader-single-nq-base')
model = model.to(cfg.device)

In [ ]:
dataset, examples, features = load_and_cache_examples(cfg, tokenizer, mode_or_filename=result_filename, output_examples=True)
eval_sampler = SequentialSampler(dataset)
eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=cfg.eval_batch_size)

In [ ]:
def evaluate(model, tokenizer):
    print("***** Running evaluation *****")
    print("  Num examples = ", len(dataset))
    print("  Batch size = ", cfg.eval_batch_size)
    all_results = []
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(cfg.device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            feature_indices = batch[3]
            outputs = model(**inputs)

        for i, feature_index in enumerate(feature_indices):
            eval_feature = features[feature_index.item()]
            unique_id = int(eval_feature.unique_id)

            start_logits = outputs.start_logits[i]
            end_logits = outputs.end_logits[i]
            result = SquadResult(unique_id, start_logits, end_logits)

            all_results.append(result)
            
    predictions = compute_predictions_logits(examples,
                                            features,
                                            all_results,
                                            cfg.n_best_size,
                                            cfg.max_answer_length,
                                            True,
                                            None,
                                            None,
                                            None,
                                            cfg.verbose_logging,
                                            False,
                                            cfg.null_score_diff_threshold,
                                            tokenizer,)
    results = squad_evaluate(examples, predictions)
    return results

In [ ]:
results = evaluate(model, tokenizer)

for key, value in results.items():
    print(f"{mode} eval_{}: {}".format(key, value))